# Download Landsat 7-9 image subsets over an AOI using AWS

_Last modified 2023-11-1._

This script is run to download Landsat images over the glaciers available through the Amazon Web Services (AWS) s3 bucket. The workflow is streamlined to analyze images for 10s to 100s of glaciers, specifically, the marine-terminating glaciers along the periphery of Greenland. Sections of code that may need to be modified are indicated as below:

    ##########################################################################################

    code to modify

    ##########################################################################################

 
### First, configure your AWS profile to access the Landsat images on the s3 bucket:

Follow instructions at https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html to get required __aws command line software__.

Set up your AWS profile with a payment account. Then configure it to your machine following these steps:

    aws configure --profile terminusmapping
    
Enter in your credentials, which will be stored locally on your computer. The Boto3 package will be used to access your credentials without leaking your keys. **Protect your AWS access keys!** DO NOT print anything that involves your ACCESS_KEY and SECRET_KEY. GitGuardian may help track any leaked keys. In order to use these credentials to download subsets of images from AWS using vsi3, **GDAL version 3.2. or newer must be installed**.

In [89]:
# AWS settings
from rasterio.session import AWSSession
import boto3
import boto3.session

cred = boto3.Session(profile_name='terminusmapping').get_credentials()
ACCESS_KEY = cred.access_key
SECRET_KEY = cred.secret_key
SESSION_TOKEN = cred.token  ## optional


s3client = boto3.client('s3', 
                        aws_access_key_id = ACCESS_KEY, 
                        aws_secret_access_key = SECRET_KEY, 
#                         aws_session_token = SESSION_TOKEN
                       )

######################################################################################
# path to the collection on AWS usgs-landsat s3 bucket:
collectionpath = 'collection02/level-1/standard/' # collection 2 level 1 data being used (contains B8)
######################################################################################

# 1) Set-up: import packages and set paths


In [90]:
import numpy as np
import pandas as pd
import scipy
import math
import subprocess
import os
import shutil
import datetime
import cv2
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob

# geospatial packages
import fiona
import geopandas as gpd
from shapely.geometry import Polygon, Point, LineString
import shapely
from matplotlib.pyplot import imshow
import rasterio as rio

# Enable fiona KML file reading driver
fiona.drvsupport.supported_drivers['KML'] = 'rw'

# distance function
def distance(x1, y1, x2, y2):
    dist = (((x2-x1)**2)+((y2-y1)**2))**(1/2)
    return dist

### Define paths, satellites, geographic projections:

In [147]:
######################################################################################
# ADJUST THESE VARIABLES:
basepath = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/' # folder containing the all glacier shapefile(s)
downloadpath ='/Volumes/SURGE_DISK/LS_MZ/' # folder to eventually contain downloaded Landsat images

sats = ['L8','L9'] # names of landsats to download images from ('L7' for Landsat 7 or 'L8' or both)
# sats = ['L7']
L9_yrs = np.arange(2019,2023).astype(str) 
L8_yrs = np.arange(2019,2023).astype(str) # set target years for L8: 2013-2021
# L7_yrs = np.arange(2012,2013).astype(str) # set target years for L7: 1999-2003
L8_bands = [8] # panchromatic band for L8
L7_bands = [8] # panchromatic band for L7
L9_bands = [8]
######################################################################################

In [148]:
## Read in AOI shapefile and determine source_srs ####################################
AOIpath = '/Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/MZ/MZ_Box_WGS.shp'
# this path is also where the UTM reprojections of the AOI shp will be stored
AOI = fiona.open(AOIpath).next()
aoi = AOI['geometry']['coordinates'][0]
print(aoi)

source_srs = '4326' # EPSG code for the current projection of the glacier shapefile(s)

# Reproject to WGS if in a different srs
# boxespath = basepath+"Box"+BoxID+"/Box"+BoxID # access the BoxID folders created 
# # construct the gdal command
# rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:"+source_srs+" "
# rp +=boxespath+"_WGS.shp "+boxespath+".shp"
# print("Command:", rp) # check command
# subprocess.run(rp, shell=True, check=True) # run the command on terminal

# # if an error is produced, check the error output on the terminal window that runs this notebook
######################################################################################

[(72.27182448672325, 38.69963864785173), (72.27598755447261, 38.6126910837067), (72.12025401244651, 38.60800602108923), (72.11590328912425, 38.69494089314537), (72.27182448672325, 38.69963864785173)]


/var/folders/mr/7f69f7y50zj85n420pyvh8cr0000gq/T/ipykernel_50576/519349794.py:4: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  AOI = fiona.open(AOIpath).next()



# 2) Find all the Landsat footprints that overlap the glaciers

This step requires the **WRS-2_bound_world_0.kml** file containing the footprints of all the Landsat scene boundaries available through the USGS (https://www.usgs.gov/media/files/landsat-wrs-2-scene-boundaries-kml-file). Place this file in your base directory (basepath). 

To check if they overlap the glacier terminus box shapefiles, the box shapefiles must be in WGS84 coordinates (ESPG: 4326). If they are not yet, we use the following GDAL command to reproject them into WGS84:

        ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:NEW_EPSG_NUMBER -s_srs EPSG:OLD_EPSG_NUMBER out.shp in.shp

In [149]:
######################################################################################
# open the kml file with the Landsat path, row footprints:
WRS = fiona.open(basepath+'WRS-2_bound_world_0.kml', driver='KML') # check the path to the world bounds file
print('Landsat footprint file opened.')
######################################################################################

Landsat footprint file opened.


In [150]:
# Grab the WGS84 coordinates of the AOI
points = [] # to hold the box vertices
# read coordinates and convert to a shapely object
for coord_pair in list(aoi): 
    lat = coord_pair[0]; lon = coord_pair[1]        
    point = shapely.geometry.Point(lat, lon) # create shapely point 
    points.append(point) # append to points list
print("coordinates recorded.") # keep track of progress

coordinates recorded.


In [151]:
paths = []; rows = []; boxes = [] # create lists to hold the paths and rows and BoxIDs

#loop through all Landsat scenes (path, row footprints)
for feature in WRS:
    # create shapely polygons from the Landsat footprints
    coordinates = feature['geometry']['coordinates'][0]
    coords = [xy[0:2] for xy in coordinates]
    pathrow_poly = Polygon(coords)
    
    # grab the path and row name from the WRS kml file:
    pathrowname = feature['properties']['Name']  
    path = pathrowname.split('_')[0]; row = pathrowname.split('_')[1]
#     print(path, row)
      
    box_points_in = 0 # counter for number of box_points in the pathrow_geom:
    for i in range(0, len(points)):
        point = points[i]
        if point.within(pathrow_poly): # if the pathrow shape contains the point
            box_points_in = box_points_in+1 # append the counter
    if box_points_in == 5: # if all box vertices are inside the footprint, save the path, row, BoxID
        paths.append('%03d' % int(path))
        rows.append('%03d' % int(row))

# Store in dataframe
boxes_pr_df = pd.DataFrame(list(zip(paths, rows)), columns=['Path', 'Row'])
boxes_pr_df # display

,Path,Row
0,152,033
1,151,033


# 3) Download metadata files from AWS s3 for overlapping Landsat scenes
     
The syntax for listing the Collection 2 Landsat image files AWS s3 bucket is as follows:

    aws s3 ls --profile terminusmapping --request-payer requester s3://usgs-landsat/collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_LS2R_pathrow_yyyyMMdd_yyyyMMdd_02_T1/ 
    
__NOTE: Including the --request-payer requester as part of this line indicates that the referenced user will be charged for data download.__

We can use the paths and rows in the dataframe to access the full Landsat scene list and the corresponding metdata files. Read https://docs.opendata.aws/landsat-pds/readme.html to learn more.
    
The metadata files will be downloaded into folders corresponding to the Landsat footprint, identified by the Path Row numbers:
    
    aws s3api get-object --bucket usgs-landsat --key collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt  --request-payer requester LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt

In [152]:
# Loop through the dataframe containing overlapping path, row info:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; folder_name = 'Path'+p+'_Row'+r+'_c2' # folder name
    bp_out = downloadpath+folder_name+'/' # output path for the downloaded files
    print("Downloaded metadata files are stored in:",bp_out)
    
    # create Path_Row folders if they don't exist already
    if os.path.exists(bp_out):
        print(folder_name, " exists already, skip directory creation")
    else:
        os.mkdir(bp_out)
        print(folder_name+" directory made")
    
    for sat in sats: # for each satellite
        if sat == 'L8':
            collectionfolder = 'oli-tirs/'; years = L8_yrs; prefix='LC08' # set folder, years, file prefix
        elif sat == 'L7':
            collectionfolder = 'etm/'; years = L7_yrs; prefix='LE07' # set folder, years, file prefix
        elif sat == 'L9':
            collectionfolder = 'oli-tirs/'; years = L8_yrs; prefix='LC09' # set folder, years, file prefix
            
        # loop through years
        for year in years:
            # grab list of images in each year, path, row folder
            find_imgs = 'aws s3 ls --profile terminusmapping --request-payer requester s3://usgs-landsat/'
            find_imgs += collectionpath+collectionfolder
            find_imgs += year+'/'+p+'/'+r+'/'

            if subprocess.run(find_imgs,shell=True).returncode != 0:
                print('No results found for '+collectionpath+collectionfolder+year+'/'+p+'/'+r+'/')
                results = [] # empty results
            else:
                result = subprocess.check_output(find_imgs,shell=True) # grab the avilable images
                results = result.split() # split string
            
            imagenames = []
            for line in results: # loop through strings
                line = str(line)
                if prefix in line and 'T1' in line: # find just the Tier-1 images
                    imgname = line[2:-2]; imagenames.append(imgname)

            # download the metadata (MTL.txt) file if it doesn't exist
            for imgname in imagenames:
                if not os.path.exists(bp_out+imgname+'_MTL.txt'): # check in output directory
                    command = 'aws s3api get-object --bucket usgs-landsat --key '+collectionpath+collectionfolder
                    command += year+'/'+p+'/'+r+'/'
                    command += imgname+'/'+imgname+'_MTL.txt'
                    command += ' --profile terminusmapping --request-payer requester '
                    command += bp_out+imgname+'_MTL.txt'
                    print('Downloading', imgname+'_MTL.txt')
                    subprocess.run(command,shell=True,check=True)
                else:
                    print(imgname+'_MTL.txt exists. Skip.')

Downloaded metadata files are stored in: /Volumes/SURGE_DISK/LS_MZ/Path152_Row033_c2/
Path152_Row033_c2 directory made
                           PRE LC08_L1GT_152033_20190116_20200829_02_T2/
                           PRE LC08_L1GT_152033_20190524_20200828_02_T2/
                           PRE LC08_L1TP_152033_20190201_20200829_02_T1/
                           PRE LC08_L1TP_152033_20190217_20200829_02_T2/
                           PRE LC08_L1TP_152033_20190305_20200829_02_T1/
                           PRE LC08_L1TP_152033_20190321_20200829_02_T1/
                           PRE LC08_L1TP_152033_20190406_20200829_02_T1/
                           PRE LC08_L1TP_152033_20190422_20200829_02_T1/
                           PRE LC08_L1TP_152033_20190508_20200828_02_T1/
                           PRE LC08_L1TP_152033_20190609_20200828_02_T1/
                           PRE LC08_L1TP_152033_20190625_20200827_02_T1/
                           PRE LC08_L1TP_152033_20190711_20200827_02_T1/
     

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-08-25T04:37:53+00:00",
    "ContentLength": 12352,
    "ETag": "\"cde1d372127310bd862554026016e882\"",
    "VersionId": "gMi2AHsql8QnSsb.iiRugZ048dk4k2db",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-11-07T04:17:54+00:00",
    "ContentLength": 12294,
    "ETag": "\"2955e1b9bb2f7d1c25e4f93280b5865e\"",
    "VersionId": "56l_Pngq2oo26imv_YYgPRsw2IHdPY0D",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
                           PRE LC08_L1GT_152033_20200103_20200823_02_T2/
                           PRE LC08_L1GT_152033_20201118_20210315_02_T2/
                           PRE LC08_L1GT_152033_20201204_20210313_02_T2/
                           PRE LC08_L1TP_152033_20200119_20200823_02_T1/
                           PRE LC08_L1TP_152033_20200204_20200823_

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-11-05T22:51:15+00:00",
    "ContentLength": 12366,
    "ETag": "\"e885f73819375b3983176901b68f3de1\"",
    "VersionId": "ql.P_uGjFreA0n31xZRfTJcbiY06eD3k",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2021-04-27T00:14:14+00:00",
    "ContentLength": 12368,
    "ETag": "\"ba6559bb880ed386cd4d3dc77ca7c621\"",
    "VersionId": "DOBu185Mly.TmwXeRTvIP1Arl2JCv9rA",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2021-03-10T03:30:17+00:00",
    "ContentLength": 12295,
    "ETag": "\"f133131baefe3520ca7e85df4b9b07eb\"",
    "VersionId": "0N3.73ipgKp3l9ER13xkiCywU8Ctce.1",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
                           PRE LC08_L

{
    "AcceptRanges": "bytes",
    "LastModified": "2021-09-25T08:34:27+00:00",
    "ContentLength": 12367,
    "ETag": "\"e88e7fe35b7bcf48425652af08c460f2\"",
    "VersionId": "EzYDRyn4yN0iLnlr89DqmT47rwaP36ve",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2021-10-14T13:56:59+00:00",
    "ContentLength": 12368,
    "ETag": "\"8dd4037bb6accb26a2aa95787201fa43\"",
    "VersionId": "F8nIpf.vQAJnCo07CkqTOgc7AOay8nxj",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2021-10-27T21:13:46+00:00",
    "ContentLength": 12367,
    "ETag": "\"56daa2da327694a2084153af0fbef551\"",
    "VersionId": "QSle6PR9WhODNZkv57sijHFcCzdQ1C9L",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "Acc

{
    "AcceptRanges": "bytes",
    "LastModified": "2022-05-04T03:18:19+00:00",
    "ContentLength": 12368,
    "ETag": "\"d370d1951e4630f6744b99aa6eda4bdd\"",
    "VersionId": "rrLt2horlAnjQAC5Zw9sCELLv0C.hCGj",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2022-05-24T22:25:10+00:00",
    "ContentLength": 12367,
    "ETag": "\"60d6d2254fef7973e5fb79fd2adda033\"",
    "VersionId": "PDBf2ONCWFjaK00OnAHXIW2kQeTajJbv",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2022-06-09T20:55:24+00:00",
    "ContentLength": 12368,
    "ETag": "\"e2266b0e9ea3b2c64aafd1de8f9fbac8\"",
    "VersionId": "SEvReQX8vVZqkmvmwueD.OrAcqkLynG5",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "Acc

...

{
    "AcceptRanges": "bytes",
    "LastModified": "2021-07-13T18:19:35+00:00",
    "ContentLength": 12366,
    "ETag": "\"245abdb285bb998ccae15ebcd3ac1a10\"",
    "VersionId": "ljh6sU5rKs5cY7Mvy5tFQ2DxpX99ta3N",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2021-08-03T22:27:49+00:00",
    "ContentLength": 12368,
    "ETag": "\"18e4f52ebe9c447839df6d9cfafe4411\"",
    "VersionId": "jeQin2VCTV.e15jrVraCVLVJx8OReIZz",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2021-08-19T06:57:03+00:00",
    "ContentLength": 12367,
    "ETag": "\"762eab5f074d1d770e9716dd54b670b0\"",
    "VersionId": "1dhMe4UyHI0evJBmeRK0qi9iHW8mChta",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "L

{
    "AcceptRanges": "bytes",
    "LastModified": "2022-02-12T05:46:46+00:00",
    "ContentLength": 12367,
    "ETag": "\"02c6c75f99ed4208e13a98906b5e0f55\"",
    "VersionId": "ut.70AjMpF2kx9isDw7QiceiiIaZp1p4",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2022-03-01T17:28:03+00:00",
    "ContentLength": 12368,
    "ETag": "\"5bb6a8858541f5c8b245e4444044a4a3\"",
    "VersionId": "hhchxbMBbeNrjfc9bUZK1duBXQKP9eO7",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2022-03-14T22:47:00+00:00",
    "ContentLength": 12294,
    "ETag": "\"fc9a013a059e51555130109046e9f2e2\"",
    "VersionId": "Yq6TwgbcPXjDw52d39sxymo2zR5nTYKf",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "Acc

                           PRE LC08_L1TP_151033_20190109_20200829_02_T1/
                           PRE LC08_L1TP_151033_20190125_20200830_02_T1/
                           PRE LC08_L1TP_151033_20190210_20200829_02_T1/
                           PRE LC08_L1TP_151033_20190226_20200829_02_T1/
                           PRE LC08_L1TP_151033_20190314_20200829_02_T1/
                           PRE LC08_L1TP_151033_20190330_20200829_02_T1/
                           PRE LC08_L1TP_151033_20190415_20200829_02_T1/
                           PRE LC08_L1TP_151033_20190501_20200829_02_T1/
                           PRE LC08_L1TP_151033_20190517_20200828_02_T1/
                           PRE LC08_L1TP_151033_20190602_20200828_02_T1/
                           PRE LC08_L1TP_151033_20190618_20200827_02_T1/
                           PRE LC08_L1TP_151033_20190704_20200827_02_T1/
                           PRE LC08_L1TP_151033_20190720_20200827_02_T1/
                           PRE LC08_L1TP_151033_201

{
    "AcceptRanges": "bytes",
    "LastModified": "2023-05-02T12:53:09+00:00",
    "ContentLength": 12208,
    "ETag": "\"0011d4251628302792505d0763ed32f6\"",
    "VersionId": "r80xFaIOKjMJdpDhO8Kyd.9Ob6xVclNZ",
    "ContentType": "binary/octet-stream",
    "ServerSideEncryption": "AES256",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2023-04-30T13:23:53+00:00",
    "ContentLength": 12207,
    "ETag": "\"1570b4a2d6a0922aa203893ee73b7d10\"",
    "VersionId": "NyE2SA8Z4sPrYoj2Jp1zvq256TILX985",
    "ContentType": "binary/octet-stream",
    "ServerSideEncryption": "AES256",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2023-04-28T03:35:24+00:00",
    "ContentLength": 12134,
    "ETag": "\"85f4252b7748073e53e505bb1a886ece\"",
    "VersionId": "XGHlanXY2eKLGz1Wa89Z9_uaPQpcuyVi",
    "ContentType": "binary/octet-stream",
    "Serv

{
    "AcceptRanges": "bytes",
    "LastModified": "2023-03-20T09:13:27+00:00",
    "ContentLength": 12208,
    "ETag": "\"9c492f28005e2cfed63bbe504cec6533\"",
    "VersionId": "n5YultAkx8ZGtGSd0pROjSmsnBYvd9Pv",
    "ContentType": "binary/octet-stream",
    "ServerSideEncryption": "AES256",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2023-03-18T08:23:26+00:00",
    "ContentLength": 12208,
    "ETag": "\"ad8160cf1ef020ce8de38d5c4b2fe7dc\"",
    "VersionId": "5TEN5Fy6esXrfrnqX3UrjvvPuVCEqOii",
    "ContentType": "binary/octet-stream",
    "ServerSideEncryption": "AES256",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2023-03-16T20:44:16+00:00",
    "ContentLength": 12206,
    "ETag": "\"4f745e2a2d9573589186dc2031320e75\"",
    "VersionId": "c3JU_TlNFIMIY3rwkLN4m4paRLPfgcUC",
    "ContentType": "binary/octet-stream",
    "Serv

# 4) Download QAPIXEL band over terminus box to determine cloud cover

If the terminus box shapefiles were not originally in UTM projection, will need to reproject them into UTM to match the Landsat projection. The code automatically finds the UTM zones from the metadata files and fills in the following syntax to reproject:
    
    ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326zone output.shp input.shp
    
#### If the terminus box shapefiles are already in UTM projection, skip the following cell and rename the files to end with "\_UTM\_##.shp" where ## corresponds to the zone number (e.g., "\_UTM\_07.shp", "\_UTM\_21.shp").

In [153]:
# If pyproj is not showing up:
os.environ['PROJ_LIB'] = '/Users/jukesliu/opt/anaconda3/envs/autoterm_env/share/proj'

In [155]:
zones = {} # initialize dictionary to hold UTM zone for each Landsat scene path row
zone_list = [] # list of zones

# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; folder_name = 'Path'+p+'_Row'+r+'_c2' # Landsat path and row
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    
    if len(os.listdir(pr_folderpath)) > 0: # if there are files in the folder
        # grab UTM Zone from the first metadata file
        mtl_scene = glob.glob(pr_folderpath+'*_MTL.txt')[0]
        mtl = open(mtl_scene, 'r')
        
        # loop through lines in the metadata file to find the UTM ZONE
        for line in mtl:  
            variable = line.split("=")[0] # grab the variable name
            if ("UTM_ZONE" in variable):
                zone = '%02d' % int(line.split("=")[1][1:-1]) # grab the 2-digit zone number
                zones.update({folder_name: zone}); zone_list.append(zone) # add to zone lists
                break
                
        # reproject shapefile(s) into UTM
        zone = zones[folder_name]
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" '+AOIpath[:-4]+'_UTM_'+zone+'.shp '+AOIpath
        rp_shp += ' -t_srs EPSG:326'+zone
        subprocess.run(rp_shp, shell=True,check=True)
        
    else: # if no files in folder, zone = nan, must fill in manually
        zone_list.append(np.nan)
        
boxes_pr_df['Zone'] = zone_list # add to the path row dataframe
boxes_pr_df.head()

,Path,Row,Zone
0,152,033,42
1,151,033,43


Use GDAL and __vsi3__ link to download subset of the quality band we will use to determine cloud cover over the terminus:

    gdalwarp -cutline path_to_shp.shp -crop_to_cutline /vsi3/usgs-landsat/collection02/level-1/standard/oli-tirs/yyyy/path/row/scene/scene_QA_PIXEL.TIF path_to_subset_QA_PIXEL.TIF


In [156]:
# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; zone = row['Zone'] # grab path, row, zone
    folder_name = 'Path'+p+'_Row'+r+'_c2'
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    pathtoshp_rp = AOIpath[:-4]+'_UTM_'+zone # path to the UTM projected box shapefile

    files = os.listdir(pr_folderpath) # grab the names of the Landsat scenes
    
    # for all files in the path row folders
    for file in files:
        scene = file[:40] # slice the filename to grab the scene name

        if scene.startswith('L') and 'T1' in scene: # L1TP scenes
            scene_year = scene[17:21] # grab the year from the scene name
            
            if scene.startswith('LC08') or scene.startswith('LC09'):
                collectionfolder='oli-tirs/'
            elif scene.startswith('LE07'):
                collectionfolder='etm/'
                
            # set path to the QA pixel Landsat files
            pathtoQAPIXEL='/vsis3/usgs-landsat/'+collectionpath+collectionfolder
            pathtoQAPIXEL+=scene_year+'/'
            pathtoQAPIXEL+=p+'/'+r+'/'
            pathtoQAPIXEL+=scene+'/'+scene+"_QA_PIXEL.TIF"
            
            # set path to the subset QA pixel files inside the path row folders
            subsetout = pr_folderpath+scene+'_QA_PIXEL.TIF' 
            
            # if the file hasn't already been downloaded
            if not os.path.exists(subsetout):
                print('Downloading', scene)
                # construct download command
                QAPIXEL_dwnld_cmd='gdalwarp -overwrite -cutline '+pathtoshp_rp+'.shp -crop_to_cutline '
                QAPIXEL_dwnld_cmd+= pathtoQAPIXEL+' '+subsetout
                QAPIXEL_dwnld_cmd+=' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2'
                QAPIXEL_dwnld_cmd+=' --config AWS_SECRET_ACCESS_KEY '+SECRET_KEY
                QAPIXEL_dwnld_cmd+=' --config AWS_ACCESS_KEY_ID '+ACCESS_KEY

                subprocess.run(QAPIXEL_dwnld_cmd, shell=True, check=True)

Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190201_20200829_02_T1/LC08_L1TP_152033_20190201_20200829_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190305_20200829_02_T1/LC08_L1TP_152033_20190305_20200829_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190321_20200829_02_T1/LC08_L1TP_152033_20190321_20200829_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1T

Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200408_20200822_02_T1/LC08_L1TP_152033_20200408_20200822_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200424_20200822_02_T1/LC08_L1TP_152033_20200424_20200822_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200510_20200820_02_T1/LC08_L1TP_152033_20200510_20200820_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1T

Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/152/033/LC08_L1TP_152033_20210716_20210721_02_T1/LC08_L1TP_152033_20210716_20210721_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/152/033/LC08_L1TP_152033_20210801_20210810_02_T1/LC08_L1TP_152033_20210801_20210810_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/152/033/LC08_L1TP_152033_20210817_20210827_02_T1/LC08_L1TP_152033_20210817_20210827_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/152/033/LC08_L1T

Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC08_L1TP_152033_20220820_20220923_02_T1/LC08_L1TP_152033_20220820_20220923_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC08_L1TP_152033_20220905_20220914_02_T1/LC08_L1TP_152033_20220905_20220914_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC08_L1TP_152033_20220921_20220929_02_T1/LC08_L1TP_152033_20220921_20220929_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC08_L1T

Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC09_L1TP_152033_20220812_20230402_02_T1/LC09_L1TP_152033_20220812_20230402_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC09_L1TP_152033_20220828_20230331_02_T1/LC09_L1TP_152033_20220828_20230331_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC09_L1TP_152033_20220913_20230329_02_T1/LC09_L1TP_152033_20220913_20230329_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 474P x 353L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/152/033/LC09_L1T

Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/151/033/LC08_L1TP_151033_20190922_20200826_02_T1/LC08_L1TP_151033_20190922_20200826_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/151/033/LC08_L1TP_151033_20191008_20200825_02_T1/LC08_L1TP_151033_20191008_20200825_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/151/033/LC08_L1TP_151033_20191024_20200825_02_T1/LC08_L1TP_151033_20191024_20200825_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/151/033/LC08_L1T

Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/151/033/LC08_L1TP_151033_20201026_20201106_02_T1/LC08_L1TP_151033_20201026_20201106_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/151/033/LC08_L1TP_151033_20201111_20210316_02_T1/LC08_L1TP_151033_20201111_20210316_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/151/033/LC08_L1TP_151033_20201127_20210316_02_T1/LC08_L1TP_151033_20201127_20210316_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/151/033/LC08_L1T

Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC08_L1TP_151033_20211114_20211125_02_T1/LC08_L1TP_151033_20211114_20211125_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC08_L1TP_151033_20211130_20211209_02_T1/LC08_L1TP_151033_20211130_20211209_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC08_L1TP_151033_20211216_20211223_02_T1/LC08_L1TP_151033_20211216_20211223_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1T

Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20221203_20221212_02_T1/LC08_L1TP_151033_20221203_20221212_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20221219_20230113_02_T1/LC08_L1TP_151033_20221219_20230113_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC09_L1TP_151033_20211109_20230506_02_T1/LC09_L1TP_151033_20211109_20230506_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC09_L1T

Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20221125_20230320_02_T1/LC09_L1TP_151033_20221125_20230320_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20221211_20230318_02_T1/LC09_L1TP_151033_20221211_20230318_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 454P x 324L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20221227_20230316_02_T1/LC09_L1TP_151033_20221227_20230316_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


# 5) Download non-cloudy Landsat images from AWS

To remove cloudy images, we will find the number of pixels in our terminus box that exceed a threshold value in the QA_PIXEL band corresponding to cloud and cloud shadow likelihood. If the fraction of cloudy pixels with values is above the threshold, we won't download the image. See Landsat Collection 2 Level 2 Science Product Guide for [Landsat 8](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat-8-9-C2-L2-ScienceProductGuide-v4.pdf) and [Landsat 7](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/LSDS-1618_Landsat-4-7_C2-L2-ScienceProductGuide-v3.pdf) more information on how the QA_PIXEL threshold values are chosen.

Additionally, we remove images that are primarily black (fill value of 1 in QA_PIXEL band). This ensures that the scenes that cut off halfway across the glacier are not included in further analysis. The fill percent threshold may need to be adjusted.

In [157]:
######################################################################################
# These are the recommended values based on the Collection 2 Level 2 Science Product Guide.
# Adjust thresholds here:
QAPIXEL_thresh_lower_L7 = 5696.0 # minimum QA pixel value threshold to be considered cloud for L7 images
QAPIXEL_thresh_upper_L7 = 7568.0 # maximum QA pixel value threshold to be considered cloud for L7 images

# Landsat 8 requires two lower thresholds, we selct between 22280 and 24472 and above 54596
QAPIXEL_thresh_lower_L8 = 22080.0 # minimum QA pixel value threshold to be considered cloud for L8 images
QAPIXEL_thresh_upper_L8 = 30048.0 # maximum QA pixel value threshold to be considered cloud for L8 images
QAPIXEL_thresh2_lower_L8 = 54596.0 # 2nd minimum QA pixel value threshold to be considered cloud for L8 images

cpercent_thresh = 100.0 # maximum cloud cover % in terminus box
fpercent_thresh = 20.0 # maximum fill % in terminus box
######################################################################################

In [158]:
# Download images that pass these thresholds:
for index, row in boxes_pr_df.iterrows():
    # grab paths
    p = row['Path']; zone = row['Zone']; r = row['Row']
    folder_name = 'Path'+p+'_Row'+r+'_c2'
    pr_folderpath = downloadpath+folder_name+'/'
    bp_out = downloadpath
    
    print(pr_folderpath)
    
    # path to the shapefile covering the region that will be downloaded
    pathtobuffer = AOIpath[:-4]+'_UTM_'+zone+'.shp' # just the box
    
    for scene in os.listdir(pr_folderpath):
        if scene.startswith('L') and scene.endswith(".TIF") and 'T1' in scene and 'L1TP' in scene: # For Tier-1 images
            scene = scene[:40] # scene name
            year = scene[17:21] # grab acquisition year
            
            QApixelpath = pr_folderpath+scene+'_QA_PIXEL.TIF' # path to QA_PIXEL file
            subsetQApixel = mpimg.imread(QApixelpath) # read in QAPIXEL file as numpy array
            totalpixels = subsetQApixel.shape[0]*subsetQApixel.shape[1] # count total number of pixels
            
            if scene.startswith("LC08") or scene.startswith('LC09'): # Landsat 8 or Landsat 9
                collectionfolder = 'oli-tirs/'; bands = L8_bands; 
                # countcloudy pixels based on thresholds:
                cloudQApixel = subsetQApixel[((subsetQApixel >= QAPIXEL_thresh_lower_L8) &
                                             (subsetQApixel < QAPIXEL_thresh_upper_L8) | 
                                             (subsetQApixel >= QAPIXEL_thresh2_lower_L8))]
                
            elif scene.startswith("LE07"): # Landsat 7
                collectionfolder = 'etm/'; bands = L7_bands
                # countcloudy pixels based on thresholds:
                cloudQApixel = subsetQApixel[((subsetQApixel >= QAPIXEL_thresh_lower_L7) & 
                                             (subsetQApixel < QAPIXEL_thresh_upper_L7))]
 
            # calculate percentages of cloud and fill pixels
            fillQApixel = subsetQApixel[subsetQApixel < 2.0] # fill pixels (value = 0 or 1)
            cloudpixels = len(cloudQApixel); fillpixels = len(fillQApixel) # count the cloudy and fill pixels
            cloudpercent = int(float(cloudpixels)/float(totalpixels)*100) # calculate percent cloudy
            fillpercent = int(float(fillpixels)/float(totalpixels)*100) # calculate percent fill
            
            # evaluate thresholds
            if cloudpercent <= cpercent_thresh and fillpercent <= fpercent_thresh:
                # download the bands for that scene into your scene folders:
                for band in bands:
                        band = str(band) # string format
                        
                        # input path to your bands in AWS:
                        pathin = '/vsis3/usgs-landsat/'+collectionpath+collectionfolder+year+'/'+p+"/"+r+"/"+scene+"/"+scene+"_B"+band+".TIF"
                        
                        outfilename = scene+"_B"+band+'.TIF' # output file name
                        pathout = downloadpath+outfilename # full output file path
                        
                        # if the file hasn't already been downloaded
                        if not os.path.exists(pathout):
                            # download
                            download_cmd = 'gdalwarp -overwrite -cutline '+pathtobuffer+' -crop_to_cutline '+pathin+' '+pathout
                            download_cmd+=' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2'
                            download_cmd+=' --config AWS_SECRET_ACCESS_KEY '+SECRET_KEY
                            download_cmd+=' --config AWS_ACCESS_KEY_ID '+ACCESS_KEY   
                            print('Downloading:', outfilename)
                            subprocess.run(download_cmd, shell=True, check=True)
                        else:
                            print(outfilename, 'exists')
            else:
                print(scene, 'failed cloud & fill thresholds: Cloud % ', cloudpercent, 'Fill %', fillpercent)

/Volumes/SURGE_DISK/LS_MZ/Path152_Row033_c2/
Downloading: LC08_L1TP_152033_20190201_20200829_02_T1_B8.TIF
Creating output file that is 949P x 708L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190201_20200829_02_T1/LC08_L1TP_152033_20190201_20200829_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190201_20200829_02_T1/LC08_L1TP_152033_20190201_20200829_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190201_20200829_02_T1/LC08_L1TP_152033_20190201_20200829_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_152033_20190201_20200829_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_152033_20190305_20200829_02_T1_B8.TIF
Creating output file that is 949P x 708L.
Processing /v

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_152033_20190812_20200827_02_T1_B8.TIF
Creating output file that is 949P x 708L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190812_20200827_02_T1/LC08_L1TP_152033_20190812_20200827_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190812_20200827_02_T1/LC08_L1TP_152033_20190812_20200827_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/152/033/LC08_L1TP_152033_20190812_20200827_02_T1/LC08_L1TP_152033_20190812_20200827_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_152033_20190812_20200827_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_152033_20190828_20200826_02_T1_B8.TIF
Creating output file that is 949P x 708L

Downloading: LC08_L1TP_152033_20200204_20200823_02_T1_B8.TIF
Creating output file that is 949P x 708L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200204_20200823_02_T1/LC08_L1TP_152033_20200204_20200823_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200204_20200823_02_T1/LC08_L1TP_152033_20200204_20200823_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200204_20200823_02_T1/LC08_L1TP_152033_20200204_20200823_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_152033_20200204_20200823_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_152033_20200220_20200822_02_T1_B8.TIF
Creating output file that is 949P x 708L.
Processing /vsis3/usgs-landsat/collection02/level-1/standa

Creating output file that is 949P x 708L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200713_20200912_02_T1/LC08_L1TP_152033_20200713_20200912_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200713_20200912_02_T1/LC08_L1TP_152033_20200713_20200912_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200713_20200912_02_T1/LC08_L1TP_152033_20200713_20200912_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_152033_20200713_20200912_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_152033_20200729_20200908_02_T1_B8.TIF
Creating output file that is 949P x 708L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/152/033/LC08_L1TP_152033_20200729_20200908_0

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_152033_20210206_20210302_02_T1_B8.TIF
Creating output file that is 949P x 708L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/152/033/LC08_L1TP_152033_20210206_20210302_02_T1/LC08_L1TP_152033_20210206_20210302_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/152/033/LC08_L1TP_152033_20210206_20210302_02_T1/LC08_L1TP_152033_20210206_20210302_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/152/033/LC08_L1TP_152033_20210206_20210302_02_T1/LC08_L1TP_152033_20210206_20210302_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_152033_20210206_20210302_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_152033_20210310_20210317_02_T1_B8.TIF
Creating output file that is 949P x 708L

...

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_151033_20211216_20211223_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC08_L1TP_151033_20211216_20211223_02_T1/LC08_L1TP_151033_20211216_20211223_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC08_L1TP_151033_20211216_20211223_02_T1/LC08_L1TP_151033_20211216_20211223_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/151/033/LC08_L1TP_151033_20211216_20211223_02_T1/LC08_L1TP_151033_20211216_20211223_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_151033_20211216_20211223_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_151033_20220101_20220106_02_T1_B8.TIF
Creating output file that is 909P x 651L

Downloading: LC08_L1TP_151033_20220525_20220602_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20220525_20220602_02_T1/LC08_L1TP_151033_20220525_20220602_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20220525_20220602_02_T1/LC08_L1TP_151033_20220525_20220602_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20220525_20220602_02_T1/LC08_L1TP_151033_20220525_20220602_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_151033_20220525_20220602_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_151033_20220610_20220616_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standa

Downloading: LC08_L1TP_151033_20221101_20221108_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20221101_20221108_02_T1/LC08_L1TP_151033_20221101_20221108_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20221101_20221108_02_T1/LC08_L1TP_151033_20221101_20221108_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC08_L1TP_151033_20221101_20221108_02_T1/LC08_L1TP_151033_20221101_20221108_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC08_L1TP_151033_20221101_20221108_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_151033_20221117_20221128_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standa

Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20220226_20230426_02_T1/LC09_L1TP_151033_20220226_20230426_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20220226_20230426_02_T1/LC09_L1TP_151033_20220226_20230426_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20220226_20230426_02_T1/LC09_L1TP_151033_20220226_20230426_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC09_L1TP_151033_20220226_20230426_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC09_L1TP_151033_20220330_20230423_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20220330_20230423_0

Downloading: LC09_L1TP_151033_20220821_20230402_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20220821_20230402_02_T1/LC09_L1TP_151033_20220821_20230402_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20220821_20230402_02_T1/LC09_L1TP_151033_20220821_20230402_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/151/033/LC09_L1TP_151033_20220821_20230402_02_T1/LC09_L1TP_151033_20220821_20230402_02_T1_B8.TIF to destination /Volumes/SURGE_DISK/LS_MZ/LC09_L1TP_151033_20220821_20230402_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC09_L1TP_151033_20220906_20230330_02_T1_B8.TIF
Creating output file that is 909P x 651L.
Processing /vsis3/usgs-landsat/collection02/level-1/standa

# 6) Delete all quality band files (*QA_PIXEL.TIF) to save space

These files will not be needed after the download step, so they can be removed to save space.

In [159]:
# Grab all path row folder names from boxes_pr_df:
paths = boxes_pr_df['Path']; rows = boxes_pr_df['Row']
for a in range(0, len(paths)): # look in each path row folder
    folder_name = 'Path'+paths[a]+'_Row'+rows[a]+'_c2'
    folderpath = downloadpath+folder_name+'/'

    # remove all files with QA_PIXEL in the name
    for file in os.listdir(folderpath):
        if 'QA_PIXEL' in file:
            os.remove(folderpath+file)